-----------------------
#### basic chain
--------------------------

In [1]:
from langchain import PromptTemplate, LLMChain
from langchain_openai  import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

In [2]:
# Define the prompt template
prompt = PromptTemplate.from_template("What is the capital of {country}?")

In [5]:
# Initialize the LLM and chain
llm   = ChatOpenAI()

chain = LLMChain(llm   = llm,    #model
                 prompt= prompt  # instructions to the model
                )

In [6]:
# Run the chain
result = chain.invoke({"country": "France"})
print(result)  # Output should be "Paris"

{'country': 'France', 'text': 'The capital of France is Paris.'}


example

In [7]:
prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe a company that makes {product}?"
)

In [8]:
chain = LLMChain(llm=llm, prompt=prompt)

In [9]:
product = "Nexon"

In [10]:
# Run the chain
result = chain.invoke({"product": product})
print(result)  

{'product': 'Nexon', 'text': 'NexonTech Solutions'}


**Simple sequential chain**

In [11]:
from langchain.chains import SimpleSequentialChain

In [12]:
llm_model = 'gpt-3.5-turbo'

In [15]:
llm = ChatOpenAI(temperature=0.9, model=llm_model)

# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe a company that makes {product}?"
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [16]:
# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Write a 20 words description for the following company:{company_name}"
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [18]:
overall_simple_chain = SimpleSequentialChain(chains = [chain_one, chain_two],
                                             verbose= True
                                            )

In [19]:
product = "fitbit"

In [21]:
overall_simple_chain.invoke(product)



> Entering new SimpleSequentialChain chain...
FitTech Innovations
FitTech Innovations is a cutting-edge fitness technology company that creates innovative products to help individuals reach their health and wellness goals.

> Finished chain.


{'input': 'fitbit',
 'output': 'FitTech Innovations is a cutting-edge fitness technology company that creates innovative products to help individuals reach their health and wellness goals.'}

**Exercise on Simple Sequential chain**

- Text Processing Pipeline

    - Remove punctuation from a text.
    - Convert the text to lowercase.
    - Count the number of words in the text.

In [22]:
# Remove punctuation
prompt_remove_punctuation = PromptTemplate(
    input_variables = ["text"],
    template        = "Remove all punctuation from the following text:\n{text}"
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=prompt_remove_punctuation)

In [23]:
# Convert to lowercase
prompt_to_lowercase = PromptTemplate(
    input_variables = ["text"],
    template        = "Convert the following text to lowercase:\n{text}"
)

# Chain 2
chain_two = LLMChain(llm=llm, prompt=prompt_to_lowercase)

In [24]:
# Count the words
prompt_count_words = PromptTemplate(
    input_variables =["text"],
    template        = "Count the number of words in the following text and return only the number:\n{text}"
)

# Chain 3
chain_three = LLMChain(llm=llm, prompt=prompt_count_words)

In [25]:
overall_simple_chain = SimpleSequentialChain(chains =[chain_one, chain_two, chain_three],
                                             verbose=True
                                            )

In [26]:
text = 'Modern humans arrived on the Indian subcontinent'

overall_simple_chain.invoke(text)



> Entering new SimpleSequentialChain chain...
Modern humans arrived on the Indian subcontinent
modern humans arrived on the indian subcontinent
7

> Finished chain.


{'input': 'Modern humans arrived on the Indian subcontinent', 'output': '7'}

#### Sequential chains

In [27]:
from langchain.chains.sequential import SequentialChain

In [28]:
# Define the prompt templates
prompt1 = PromptTemplate.from_template("What is the capital of {country}?")
prompt2 = PromptTemplate.from_template("Describe the main attractions in {city}.")

In [29]:
# Initialize the language model
llm = ChatOpenAI()

In [30]:
# Create individual chains with explicit output
chain1 = LLMChain(llm       = llm, 
                  prompt    = prompt1, 
                  output_key="city")               # Sets "city" as output for chain1

chain2 = LLMChain(llm       = llm, 
                  prompt    = prompt2, 
                  output_key= "city_description")  # Sets "city_description" as output for chain2

In [31]:
# Set up the SequentialChain
sequential_chain = SequentialChain(
    chains           = [chain1, chain2],
    input_variables  = ["country"],          # Input to the first chain
    output_variables = ["city_description"]  # Output from the last chain
)

In [32]:
# Run the SequentialChain
result = sequential_chain.invoke({"country": "Japan"})
print(result["city_description"])  # Should describe attractions in the capital city

Tokyo, the capital of Japan, is a vibrant and bustling metropolis that offers a diverse range of attractions for visitors to enjoy. Some of the main attractions in Tokyo include:

1. The iconic Tokyo Skytree, one of the tallest structures in the world, offers panoramic views of the city from its observation decks.

2. The historic Senso-ji Temple in Asakusa is a popular destination for tourists, with its colorful pagoda and bustling market street.

3. The bustling neighborhoods of Shibuya and Shinjuku are known for their shopping, dining, and nightlife options, as well as iconic landmarks such as the Shibuya Crossing and the Golden Gai district.

4. The Imperial Palace and its surrounding gardens offer a tranquil escape from the hustle and bustle of the city, with beautiful cherry blossoms in the spring and autumn foliage in the fall.

5. The trendy district of Harajuku is known for its quirky fashion and vibrant street art, as well as the famous Meiji Shrine and Yoyogi Park.

6. The T

**Exercise on Sequential chain with multiple input variables**

Design a Sequential Chain to analyze medical data from a patient's record. The chain should:

- Take multiple inputs: age, blood_pressure, cholesterol_level, and smoking_status.

    - Step 1: Assess the patient's `risk level` for heart disease based on the inputs (e.g., Low, Moderate, High).
    - Step 2: Suggest `lifestyle changes` based on the risk level and smoking status.
    - Step 3: Provide a `follow-up recommendation` based on the risk level and age.

In [33]:
# Step 1: Assess heart disease risk level
prompt_risk_assessment = PromptTemplate(
    input_variables = ["age", "blood_pressure", "cholesterol_level", "smoking_status"],
    template        = (
        "Based on the following information:\n"
        "- Age: {age}\n"
        "- Blood Pressure: {blood_pressure}\n"
        "- Cholesterol Level: {cholesterol_level}\n"
        "- Smoking Status: {smoking_status}\n"
        "Assess the risk level for heart disease as Low, Moderate, or High."
    )
)

In [34]:
# Step 2: Suggest lifestyle changes
prompt_lifestyle_changes = PromptTemplate(
    input_variables = ["risk_level", "smoking_status"],
    template        = (
        "Given that the heart disease risk level is {risk_level} and the patient is a "
        "{smoking_status} smoker, suggest lifestyle changes to improve health."
    )
)

In [35]:
# Step 3: Provide follow-up recommendation
prompt_follow_up = PromptTemplate(
    input_variables = ["risk_level", "age"],
    template = (
        "For a patient with a heart disease risk level of {risk_level} and age {age}, "
        "provide a follow-up recommendation (e.g., frequency of doctor visits or specific tests)."
    )
)

In [36]:
# Create individual chains with explicit output
chain1 = LLMChain(llm       = llm, 
                  prompt    = prompt_risk_assessment, 
                  output_key="risk_level")              

chain2 = LLMChain(llm       = llm, 
                  prompt    = prompt_lifestyle_changes, 
                  output_key= "lifestyle_changes")  

chain3 = LLMChain(llm       = llm, 
                  prompt    = prompt_lifestyle_changes, 
                  output_key= "recommendation")  

In [37]:
# Set up the SequentialChain
sequential_chain = SequentialChain(
    chains           = [chain1, chain2, chain3],
    input_variables  = ["age", "blood_pressure", "cholesterol_level", "smoking_status"],         
    output_variables = ["recommendation"]         # Output from the last chain
)

In [38]:
# input data
patient_data = {
    "age": 43,
    "blood_pressure": "130/100",
    "cholesterol_level": "200 mg/dL",
    "smoking_status": 'current'
}


In [39]:
# Run the SequentialChain
result = sequential_chain.invoke(patient_data)
print(result["recommendation"])

1. Quit smoking: Smoking is a major risk factor for heart disease. Quitting smoking can greatly reduce your risk of developing heart disease. Consider speaking with your healthcare provider about resources and support to help you quit.

2. Adopt a healthy diet: Focus on eating a diet rich in fruits, vegetables, whole grains, lean proteins, and healthy fats. Limit your intake of saturated fats, trans fats, cholesterol, and sodium. This can help improve your cholesterol levels and blood pressure.

3. Increase physical activity: Aim for at least 150 minutes of moderate-intensity exercise or 75 minutes of vigorous-intensity exercise each week. Regular physical activity can help lower your blood pressure, improve your cholesterol levels, and promote overall heart health.

4. Maintain a healthy weight: If you are overweight or obese, losing even a small amount of weight can help improve your heart health. Aim to achieve and maintain a healthy weight through a combination of healthy eating an

#### Router chain

In [40]:
physics_template = """
You are a very smart physics professor. You are great at answering questions about physics in a concise
and easy to understand manner. When you don't know the answer to a question you admit that you don't know.

Here is a question:
{input}
"""

math_template = """
You are a very good mathematician. You are great at answering math questions. 
You are so good because you are able to break down hard problems into their component parts, 
answer the component parts, and then put them together to answer the broader question.

Here is a question:
{input}
"""

history_template = """
You are a very good historian. You have an excellent knowledge of and understanding of people,
events and contexts from a range of historical periods. 
You have the ability to think, reflect, debate, discuss and evaluate the past. 
You have a respect for historical evidence and the ability to make use of it to support your explanations 
and judgements.

Here is a question:
{input}
"""

computerscience_template = """
You are a successful computer scientist. You have a passion for creativity, collaboration,
forward-thinking, confidence, strong problem-solving capabilities,
understanding of theories and algorithms, and excellent communication skills.
You are great at answering coding questions. 
You are so good because you know how to solve a problem by 
describing the solution in imperative steps 
that a machine can easily interpret and you know how to 
choose a solution that has a good balance between 
time complexity and space complexity. 

Here is a question:
{input}
"""

In [41]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    }
]

In [42]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.prompts import PromptTemplate

In [43]:
llm = ChatOpenAI(temperature=0, model=llm_model)

In [44]:
destination_chains = {}

for p_info in prompt_infos:
    name            = p_info["name"]
    prompt_template = p_info["prompt_template"]
    
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    
    chain  = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations     = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [45]:
print(destinations_str)

physics: Good for answering questions about physics
math: Good for answering math questions
History: Good for answering history questions
computer science: Good for answering computer science questions


In [46]:
# when LLM cant decide which chain to use
default_prompt = ChatPromptTemplate.from_template("{input}")

default_chain  = LLMChain(llm=llm, prompt=default_prompt)

In [47]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a 
language model select the model prompt best suited for the input. 

You will be given the names of the available prompts and a 
description of what the prompt is best suited for. 

You may also revise the original input if you think that revising
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string  name of the prompt to use or "DEFAULT"
    "next_inputs": string  a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [48]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)

print(router_template)

Given a raw text input to a 
language model select the model prompt best suited for the input. 

You will be given the names of the available prompts and a 
description of what the prompt is best suited for. 

You may also revise the original input if you think that revising
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{
    "destination": string  name of the prompt to use or "DEFAULT"
    "next_inputs": string  a potentially modified version of the original input
}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is notwell suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
physics: Good for answering questions about physics
math: Good for answering math questions
Histor

In [49]:
router_prompt = PromptTemplate(
    template       = router_template,
    input_variables= ["input"],
    output_parser  = RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [50]:
chain = MultiPromptChain(router_chain      =router_chain, 
                         destination_chains=destination_chains, 
                         default_chain     =default_chain, 
                         verbose           =True
                        )

C:\Users\bhupe\AppData\Local\Temp\ipykernel_27292\1422575192.py:1: LangChainDeprecationWarning: Use RunnableLambda to select from multiple prompt templates. See example in API reference: https://api.python.langchain.com/en/latest/chains/langchain.chains.router.multi_prompt.MultiPromptChain.html
  chain = MultiPromptChain(router_chain      =router_chain,


In [51]:
chain.run("What is black body radiation?")

C:\Users\bhupe\AppData\Local\Temp\ipykernel_27292\1733274077.py:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  chain.run("What is black body radiation?")




> Entering new MultiPromptChain chain...
physics: {'input': 'What is black body radiation?'}
> Finished chain.


"Black body radiation refers to the electromagnetic radiation emitted by a perfect black body, which is an idealized physical body that absorbs all incident electromagnetic radiation. The radiation emitted by a black body depends only on its temperature and follows a specific distribution known as Planck's law. This radiation is characterized by a continuous spectrum of wavelengths and intensities, with the peak intensity shifting to shorter wavelengths as the temperature of the black body increases. Black body radiation plays a crucial role in understanding various phenomena in physics, such as thermal radiation and the quantization of energy."

In [74]:
chain.run("what is 2 + 2")



> Entering new MultiPromptChain chain...
math: {'input': 'what is 2 + 2'}
> Finished chain.


'The answer to 2 + 2 is 4.'

In [52]:
chain.run("What is the capital of Karnataka state in India?")



> Entering new MultiPromptChain chain...
None: {'input': 'What is the capital of Karnataka state in India?'}
> Finished chain.


'The capital of Karnataka state in India is Bangalore.'

**Exercise problem on router chain**

- Create prompt templates for each of the three specialized chains.
- Use the Router Chain to select the appropriate specialized chain based on the `symptom description`.
- Simulate a case where the user provides a symptom, and the system correctly identifies and processes it through the appropriate specialized chain.

In [76]:
# Specialized Chain Prompts
cardiology_prompt = PromptTemplate(
    input_variables= ["symptom"],
    template       = (
        "A patient presents with the following symptom: {symptom}. "
        "As a cardiologist, provide a preliminary diagnosis and next steps."
    )
)

neurology_prompt = PromptTemplate(
    input_variables= ["symptom"],
    template       = (
        "A patient presents with the following symptom: {symptom}. "
        "As a neurologist, provide a preliminary diagnosis and next steps."
    )
)

general_medicine_prompt = PromptTemplate(
    input_variables = ["symptom"],
    template        = (
        "A patient presents with the following symptom: {symptom}. "
        "As a general physician, provide a preliminary diagnosis and next steps."
    )
)


In [77]:
# Mapping of conditions to specialized chains
condition_to_prompt = {
    "Cardiology": cardiology_prompt,
    "Neurology": neurology_prompt,
    "General Medicine": general_medicine_prompt,
}

In [78]:
# Initialize Router Chain
router_chain = MultiPromptChain.from_prompts(condition_to_prompt, llm)

TypeError: tuple indices must be integers or slices, not str

#### ConversationBufferMemory
with simple LLMChain

In [53]:
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory

In [54]:
llm = ChatOpenAI()

In [55]:
memory = ConversationBufferMemory()

C:\Users\bhupe\AppData\Local\Temp\ipykernel_27292\995385594.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory()


In [56]:
# Get user input
user_input = "Where is Bangalore?"

In [57]:
# Load previous memory (initially empty)
previous_memory = memory.load_memory_variables({})  # No input needed for initial load

In [58]:
previous_memory

{'history': ''}

In [59]:
previous_memory.get('history', "")

''

In [60]:
# Construct the context from previous interactions
context = previous_memory.get('history', "")
context

''

In [61]:
# Create a prompt using context and user input
prompt_template = PromptTemplate.from_template(
    f'''You are a helpful assistant.  
    
    Here are the previous interactions: {context.strip()}
    
    User: {user_input}
    Bot:
    '''
)

In [62]:
# Create an LLMChain
chain = LLMChain(llm=llm, prompt=prompt_template)

In [63]:
# Get response from the chain
combined_input = {"context": context.strip(), "input": user_input}
response = chain.invoke(combined_input)

In [65]:
response

{'context': '',
 'input': 'Where is Bangalore?',
 'text': 'Bangalore is located in the southern part of India, in the state of Karnataka. It is known for its pleasant climate, vibrant culture, and booming IT industry.'}

In [66]:
# Save user input and bot response to memory
# Ensure we pass the correct values to save_context
memory.save_context({"input": user_input}, {"output": str(response)})

In [67]:
import json

# Directly pretty-print the dictionary with indentation
print(json.dumps(memory.to_json(), indent=4))

{
    "lc": 1,
    "type": "not_implemented",
    "id": [
        "langchain",
        "memory",
        "buffer",
        "ConversationBufferMemory"
    ],
    "repr": "ConversationBufferMemory(chat_memory=InMemoryChatMessageHistory(messages=[HumanMessage(content='Where is Bangalore?', additional_kwargs={}, response_metadata={}), AIMessage(content=\"{'context': '', 'input': 'Where is Bangalore?', 'text': 'Bangalore is located in the southern part of India, in the state of Karnataka. It is known for its pleasant climate, vibrant culture, and booming IT industry.'}\", additional_kwargs={}, response_metadata={})]))"
}


In [68]:
# Load previous memory (initially empty)
context = dict(memory.load_memory_variables({}))  # No input needed for initial load

In [69]:
context

{'history': "Human: Where is Bangalore?\nAI: {'context': '', 'input': 'Where is Bangalore?', 'text': 'Bangalore is located in the southern part of India, in the state of Karnataka. It is known for its pleasant climate, vibrant culture, and booming IT industry.'}"}

In [70]:
# Initialize the LLM and prompt template
llm = ChatOpenAI()

In [71]:
prompt_template = PromptTemplate(input_variables= ["context", "input"], 
                                 template       = "{context}\nUser: {input}\nAI:")

In [72]:
# Initialize LLMChain with the LLM and template
chain = LLMChain(llm   = llm, 
                 prompt= prompt_template)

In [73]:
# Initialize an empty context
context = ""

In [76]:
# Example interactions
user_inputs = ["What is the capital of France?", 
               "And its population?", 
               "What about Germany?",
               "Compare the 2 citites in terms of job opportunities"
              ]

In [77]:
from pprint import pprint

In [78]:
for user_input in user_inputs:
    # Run the LLM chain, adding context
    response = chain.invoke({"context": context, "input": user_input})
    print(user_input)
    pprint(response)
    print("-----------------")

    # Update context with user input and LLM response
    context += f"\nUser: {user_input}\nAI: {response}"

What is the capital of France?
{'context': '',
 'input': 'What is the capital of France?',
 'text': 'The capital of France is Paris.'}
-----------------
And its population?
{'context': '\n'
            'User: What is the capital of France?\n'
            "AI: {'context': '', 'input': 'What is the capital of France?', "
            "'text': 'The capital of France is Paris.'}",
 'input': 'And its population?',
 'text': "{'context': '', 'input': 'And its population?', 'text': 'The "
         "population of Paris is approximately 2.1 million people.'}"}
-----------------
What about Germany?
{'context': '\n'
            'User: What is the capital of France?\n'
            "AI: {'context': '', 'input': 'What is the capital of France?', "
            "'text': 'The capital of France is Paris.'}\n"
            'User: And its population?\n'
            'AI: {\'context\': "\\nUser: What is the capital of France?\\nAI: '
            "{'context': '', 'input': 'What is the capital of France?', "
   

**Example**

In [79]:
llm = ChatOpenAI(temperature=0.1, model=llm_model)

In [80]:
# prompt template 1: translate to english
first_prompt = ChatPromptTemplate.from_template(
    "Translate the following review to english:"
    "\n\n{Review}"
)

# chain 1: input= Review and output= English_Review
chain_one = LLMChain(llm       = llm, 
                     prompt    = first_prompt, 
                     output_key= "English_Review"
                    )

In [81]:
second_prompt = ChatPromptTemplate.from_template(
    "Can you summarize the following review in 2 sentences:"
    "\n\n{English_Review}"
)

# chain 2: input= English_Review and output= summary
chain_two = LLMChain(llm       = llm, 
                     prompt    = second_prompt, 
                     output_key= "summary"
                    )

In [82]:
# prompt template 3: translate to english
third_prompt = ChatPromptTemplate.from_template(
    "What language is the following review:\n\n{Review}"
)

# chain 3: input= Review and output= language
chain_three = LLMChain(llm       = llm, 
                       prompt    = third_prompt,
                       output_key= "language"
                      )

In [83]:
# prompt template 4: follow up message
fourth_prompt = ChatPromptTemplate.from_template(
    "Write a follow up response to the following "
    "summary in the specified language:"
    "\n\nSummary: {summary}\n\nLanguage: {language}"
)

# chain 4: input= summary, language and output= followup_message
chain_four = LLMChain(llm       = llm, 
                      prompt    = fourth_prompt,
                      output_key= "followup_message"
                     )

In [84]:
# overall_chain: input= Review 
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains          = [chain_one, chain_two, chain_three, chain_four],
    input_variables = ["Review"],
    output_variables= ["English_Review", "summary","followup_message"],
    verbose         = True
)

In [85]:
review = '''
বাংলাদেশ একটি সমৃদ্ধ ইতিহাস ও সংস্কৃতির দেশ। এই দেশে রয়েছে প্রাচীন স্থাপত্য, নদী, সবুজ প্রকৃতি এবং অসংখ্য জাতিগোষ্ঠীর বৈচিত্র্য। 
বাংলাদেশের মানুষ অতিথিপরায়ণ এবং তারা তাদের ঐতিহ্য ও সংস্কৃতিকে ভালোবাসে। প্রতি বছর এখানে বিভিন্ন ধরনের উৎসব পালন করা হয়, 
যেমন পহেলা বৈশাখ, ঈদ, দুর্গাপূজা, এবং বিজয় দিবস। এই উৎসবগুলোতে মানুষ একসঙ্গে আনন্দ করে, গান গায়, নাচে এবং খাবার খায়।

বাংলাদেশের অন্যতম বড় আকর্ষণ হচ্ছে সুন্দরবন, যা পৃথিবীর বৃহত্তম ম্যানগ্রোভ বন। এখানে রয়েল বেঙ্গল টাইগার, চিত্রা হরিণ, 
বানর এবং নানা ধরনের বন্যপ্রাণীর বসবাস। সুন্দরবনের প্রকৃতি অত্যন্ত সুন্দর এবং এটি প্রতিবছর হাজার হাজার পর্যটককে আকর্ষণ করে। 
তাছাড়া কক্সবাজার বিশ্বের দীর্ঘতম সমুদ্রসৈকত হিসাবে পরিচিত। সাদা বালুর সৈকত, নীল পানি, এবং সূর্যাস্তের দৃশ্য প্রতিটি দর্শকের মনে দাগ কাটে।

বাংলাদেশের মানুষের জীবনধারা খুবই সরল এবং প্রকৃতির সাথে গভীরভাবে সংযুক্ত। বেশিরভাগ মানুষ গ্রামাঞ্চলে বাস করে এবং তারা 
কৃষিকাজে ব্যস্ত থাকে। ধান, পাট, চা, এবং মাছ বাংলাদেশের প্রধান পণ্য। দেশটি ধীরে ধীরে শিল্প এবং প্রযুক্তিতে এগিয়ে যাচ্ছে এবং এশিয়ার 
একটি গুরুত্বপূর্ণ অর্থনৈতিক কেন্দ্রে পরিণত হচ্ছে।
'''

In [86]:
overall_chain.invoke(review)



> Entering new SequentialChain chain...

> Finished chain.


{'Review': '\nবাংলাদেশ একটি সমৃদ্ধ ইতিহাস ও সংস্কৃতির দেশ। এই দেশে রয়েছে প্রাচীন স্থাপত্য, নদী, সবুজ প্রকৃতি এবং অসংখ্য জাতিগোষ্ঠীর বৈচিত্র্য। \nবাংলাদেশের মানুষ অতিথিপরায়ণ এবং তারা তাদের ঐতিহ্য ও সংস্কৃতিকে ভালোবাসে। প্রতি বছর এখানে বিভিন্ন ধরনের উৎসব পালন করা হয়, \nযেমন পহেলা বৈশাখ, ঈদ, দুর্গাপূজা, এবং বিজয় দিবস। এই উৎসবগুলোতে মানুষ একসঙ্গে আনন্দ করে, গান গায়, নাচে এবং খাবার খায়।\n\nবাংলাদেশের অন্যতম বড় আকর্ষণ হচ্ছে সুন্দরবন, যা পৃথিবীর বৃহত্তম ম্যানগ্রোভ বন। এখানে রয়েল বেঙ্গল টাইগার, চিত্রা হরিণ, \nবানর এবং নানা ধরনের বন্যপ্রাণীর বসবাস। সুন্দরবনের প্রকৃতি অত্যন্ত সুন্দর এবং এটি প্রতিবছর হাজার হাজার পর্যটককে আকর্ষণ করে। \nতাছাড়া কক্সবাজার বিশ্বের দীর্ঘতম সমুদ্রসৈকত হিসাবে পরিচিত। সাদা বালুর সৈকত, নীল পানি, এবং সূর্যাস্তের দৃশ্য প্রতিটি দর্শকের মনে দাগ কাটে।\n\nবাংলাদেশের মানুষের জীবনধারা খুবই সরল এবং প্রকৃতির সাথে গভীরভাবে সংযুক্ত। বেশিরভাগ মানুষ গ্রামাঞ্চলে বাস করে এবং তারা \nকৃষিকাজে ব্যস্ত থাকে। ধান, পাট, চা, এবং মাছ বাংলাদেশের প্রধান পণ্য। দেশটি ধীরে ধীরে শিল্প এবং প্রযু

**Example** - Conversation Buffer memory

In [87]:
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain

In [88]:
# Initialize the LLM and prompt template
llm = ChatOpenAI()

In [89]:
# Initialize memory
memory = ConversationBufferMemory()

In [90]:
# Initialize LLMChain with the LLM, template, and memory
chain = ConversationChain(llm     = llm, 
                          #prompt= prompt_template,  
                          memory  = memory, 
                          verbose = True)

C:\Users\bhupe\AppData\Local\Temp\ipykernel_27292\4024638112.py:2: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :meth:`~RunnableWithMessageHistory: https://python.langchain.com/v0.2/api_reference/core/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html` instead.
  chain = ConversationChain(llm     = llm,


In [91]:
print(chain.prompt.template)

The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{history}
Human: {input}
AI:


In [92]:
# Example interactions
user_inputs = [
    "What is the capital of China?",
    "And its population?",
    "What about India?",
    "Compare the 2 cities in terms of weather in general",
    "Which city is bigger in geographical size?",
    "How do the cultures of these places differ?",
    "What languages are predominantly spoken in these locations?",
    "What is the general cost of living like in each place?",
    "How do the healthcare systems compare between these regions?",
    "What environmental challenges do these areas face?",
    "How have these locations changed over the last few decades?"
]

In [93]:
for user_input in user_inputs:
    # Invoke the chain
    response = chain.invoke({"input": user_input})
    
    print(f"User: {user_input}")
    pprint(response)
    print("---------------------------------------------------------------------")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: What is the capital of China?
AI:

> Finished chain.
User: What is the capital of China?
{'history': '',
 'input': 'What is the capital of China?',
 'response': 'The capital of China is Beijing. It is one of the most populous '
             'cities in the world and serves as the political, cultural, and '
             'educational center of the country. Beijing is known for its '
             'historical landmarks such as the Great Wall of China, the '
             'Forbidden City, and the Temple of Heaven. It is also a major '
             'transportation hub with a well-developed public transportation '
             'system including the Beijing Subway and nu

#### counting tokens at each interaction

In [94]:
from langchain.callbacks import get_openai_callback

In [95]:
# Initialize the LLM and prompt template
llm = ChatOpenAI()

In [96]:
# Initialize memory
memory = ConversationBufferMemory()

In [97]:
# Initialize LLMChain with the LLM, template, and memory
chain = ConversationChain(llm   = llm, 
                          #prompt= prompt_template,  
                          memory= memory)

In [98]:
# Example interactions
user_inputs = [
    "What is the capital of China?",
    "And its population?",
    "What about India?",
    "Compare the 2 cities in terms of weather in general",
    "Which city is bigger in geographical size?",
    "How do the cultures of these places differ?",
    "What languages are predominantly spoken in these locations?",
    "What is the general cost of living like in each place?",
    "How do the healthcare systems compare between these regions?",
    "What environmental challenges do these areas face?",
    "How have these locations changed over the last few decades?"
]

In [99]:
for user_input in user_inputs:

    with get_openai_callback() as cb:
        # Invoke the chain
        response = chain.invoke({"input": user_input})
        
    
    print(f"User: {user_input}")
    pprint(response)
    print(f'>>>> Spent a total of {cb.total_tokens} tokens')
    print("---------------------------------------------------------------------")

User: What is the capital of China?
{'history': '',
 'input': 'What is the capital of China?',
 'response': 'The capital of China is Beijing. It is one of the most populous '
             'cities in the world and serves as the political, cultural, and '
             'educational center of the country. Beijing is known for its '
             'historic sites such as the Great Wall, Forbidden City, and '
             'Temple of Heaven. It also has a thriving arts and culinary '
             'scene, making it a popular destination for tourists from around '
             'the world.'}
>>>> Spent a total of 146 tokens
---------------------------------------------------------------------
User: And its population?
{'history': 'Human: What is the capital of China?\n'
            'AI: The capital of China is Beijing. It is one of the most '
            'populous cities in the world and serves as the political, '
            'cultural, and educational center of the country. Beijing is known '
   

#### ConversationSummaryMemory

The **ConversationSummaryMemory** addresses a limitation of the ConversationBufferMemory. As conversations progress, the token count of our context history increases, potentially reaching a point where it exceeds the token limit of the language model (LLM).

**Key Features:**
- **Summarized Conversation Storage**: Instead of keeping a raw, unmodified history of the conversation, the memory stores a **summarized** version of previous conversation snippets.
- The summarization is performed by an **LLM** (Large Language Model), ensuring the most relevant information is retained while minimizing token usage.

**Functionality:**
- This memory type stores conversation snippets in a summarized form, allowing more efficient use of tokens by condensing long conversations.
- To perform the summarization, the memory constructor needs to have access to an LLM, which will handle the summarization process.

**Use Case:**
- **ConversationSummaryMemory** is useful for longer conversations where token limits are a concern, allowing the model to maintain context while preventing excessive token usage by summarizing the dialogue.


In [100]:
from langchain.chains.conversation.memory import ConversationSummaryMemory
from langchain.chains import ConversationChain

In [101]:
# Initialize the LLM and prompt template
llm = ChatOpenAI()

In [102]:
# Initialize memory
memory = ConversationSummaryMemory(llm=llm)

C:\Users\bhupe\AppData\Local\Temp\ipykernel_27292\4291718127.py:2: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationSummaryMemory(llm=llm)


In [103]:
# Initialize LLMChain with the LLM, template, and memory
conversation_sum_chain = ConversationChain(llm   = llm, 
                          #prompt                = prompt_template,  
                          memory                 = memory)

In [104]:
# Example interactions
user_inputs = [
    "What is the capital of China?",
    "And its population?",
    "What about India?",
    "Compare the 2 cities in terms of weather in general",
    "Which city is bigger in geographical size?",
    "How do the cultures of these places differ?",
    "What languages are predominantly spoken in these locations?",
    "What is the general cost of living like in each place?",
    "How do the healthcare systems compare between these regions?",
    "What environmental challenges do these areas face?",
    "How have these locations changed over the last few decades?"
]

In [105]:
for user_input in user_inputs:

    with get_openai_callback() as cb:
        # Invoke the chain
        response = conversation_sum_chain.invoke({"input": user_input})
        
    
    print(f"User: {user_input}")
    pprint(response)
    print(f'>>>> Spent a total of {cb.total_tokens} tokens')
    print("---------------------------------------------------------------------")

User: What is the capital of China?
{'history': '',
 'input': 'What is the capital of China?',
 'response': 'The capital of China is Beijing. It is the political, cultural, '
             'and educational center of the country, as well as being one of '
             'the most populous cities in the world. Beijing is known for its '
             'historical sites such as the Great Wall, Forbidden City, and '
             'Temple of Heaven.'}
>>>> Spent a total of 393 tokens
---------------------------------------------------------------------
User: And its population?
{'history': 'The human asks what the AI thinks of artificial intelligence. The '
            'AI thinks artificial intelligence is a force for good because it '
            'will help humans reach their full potential. The human then asks '
            'the AI what the capital of China is. The AI responds that the '
            'capital of China is Beijing, known for its historical sites and '
            'being a populous

#### ConversationBufferWindowMemory

The **ConversationBufferWindowMemory** is another efficient memory type for managing conversation context. It functions like short-term memory, keeping only a set number of recent conversation exchanges while dropping older ones.

**Key Features:**
- **Windowed Raw Conversation Storage**: The memory retains the most recent conversation snippets in their **raw, unmodified form**.
- The number of stored exchanges is controlled by the `k` parameter, which defines the size of the window, i.e., how many of the latest interactions to keep in memory.
  
**Functionality:**
- The memory acts as a sliding window, keeping the last `k` interactions and dropping older ones. This reduces both the aggregate token count and the number of tokens processed per call.
- By limiting the number of exchanges stored, the model can maintain relevant context while staying within token limits.

**Use Case:**
- **ConversationBufferWindowMemory** is useful when you want to keep a recent context without overwhelming the model with the entire conversation history. It allows efficient use of tokens while retaining recent exchanges for coherent responses.


In [106]:
from langchain.chains.conversation.memory import ConversationBufferWindowMemory

In [107]:
# Initialize memory
memory = ConversationBufferWindowMemory(k=5)

C:\Users\bhupe\AppData\Local\Temp\ipykernel_27292\1355508805.py:2: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferWindowMemory(k=5)


In [ ]:
...

In [ ]:
...

----------------------------------
#### Stuff chain
-----------------------------------

In [108]:
from langchain.chains import StuffDocumentsChain
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
from langchain.schema import Document  # Import the Document class

In [121]:
# Create a prompt template for summarization
summary_prompt = PromptTemplate(input_variables= ["text"], 
                                template       = "Summarize the following text in max 1 sentence: {text}")

In [122]:
# Create an LLMChain with the LLM and the prompt template
llm_chain = LLMChain(llm=llm, prompt=summary_prompt)

In [123]:
# Initialize the Stuff Chain with the LLMChain
stuff_chain = StuffDocumentsChain(llm_chain=llm_chain)

In [124]:
# Define the documents (text chunks) to be summarized as instances of Document
documents = [
    Document(page_content="Artificial Intelligence (AI) has a profound impact on many industries, including healthcare and finance."),
    Document(page_content="Machine learning is a subset of AI that focuses on developing systems that can learn from data."),
    Document(page_content="Deep learning, a more complex form of machine learning, has enabled advancements in image and speech recognition.")
]

In [125]:
# Run the Stuff Chain to get a summary of all text chunks combined
summary = stuff_chain.invoke(input={"input_documents": documents})
summary

{'input_documents': [Document(metadata={}, page_content='Artificial Intelligence (AI) has a profound impact on many industries, including healthcare and finance.'),
  Document(metadata={}, page_content='Machine learning is a subset of AI that focuses on developing systems that can learn from data.'),
  Document(metadata={}, page_content='Deep learning, a more complex form of machine learning, has enabled advancements in image and speech recognition.')],
 'output_text': 'AI is revolutionizing various industries, with machine learning and deep learning playing crucial roles in developing systems that can learn from data and improve image and speech recognition.'}

----------------------------
#### Refine chain
---------------------------

In [126]:
from langchain.chains import RefineDocumentsChain, LLMChain
from langchain.prompts import PromptTemplate
from langchain_openai import OpenAI, ChatOpenAI
from langchain.schema import Document

In [127]:
# Initialize the LLM and prompt template
llm = ChatOpenAI()

In [128]:
# Create a prompt template for initial output
initial_prompt = PromptTemplate(
    input_variables= ["page_content"], 
    template       = "Generate a brief overview based on the following information: {page_content}"
)

In [129]:
# Create a prompt template for refining the output
refine_prompt = PromptTemplate(
    input_variables = ["existing_answer", "new_information"], 
    template        = "Refine the following answer by incorporating this new information: {new_information}. Answer: {existing_answer}"
)

In [130]:
# Create an initial LLMChain for the first response
initial_chain = LLMChain(llm=llm, prompt=initial_prompt)

# Create a refine LLMChain for the refinement process
refine_chain = LLMChain(llm=llm, prompt=refine_prompt)

In [131]:
# Combine the chains into a RefineDocumentsChain
refine_documents_chain = RefineDocumentsChain(
    initial_llm_chain    = initial_chain,       # Initial processing chain
    refine_llm_chain     = refine_chain,        # Refinement processing chain
    initial_response_name= "initial_response",  # Name for the initial response variable
    verbose              = True
)

C:\Users\bhupe\AppData\Local\Temp\ipykernel_27292\987537964.py:2: LangChainDeprecationWarning: This class is deprecated. Please see the migration guide here for a recommended replacement: https://python.langchain.com/docs/versions/migrating_chains/refine_docs_chain/
  refine_documents_chain = RefineDocumentsChain(


In [132]:
# Define the documents to refine
documents = [
    Document(page_content="Artificial Intelligence (AI) is a field that simulates human intelligence."),
    Document(page_content="Machine learning is a significant branch of AI that focuses on data analysis."),
    Document(page_content="Deep learning models, a subset of machine learning, utilize neural networks.")
]

In [133]:
# Step 1: Run the initial chain to get the summary
initial_response = initial_chain.invoke(input={"page_content": " ".join(doc.page_content for doc in documents)})

In [134]:
initial_response

{'page_content': 'Artificial Intelligence (AI) is a field that simulates human intelligence. Machine learning is a significant branch of AI that focuses on data analysis. Deep learning models, a subset of machine learning, utilize neural networks.',
 'text': 'Artificial Intelligence (AI) is a field of computer science that aims to simulate human intelligence. Machine learning, a significant branch of AI, focuses on analyzing data and making predictions based on patterns. Deep learning models, a subset of machine learning, use neural networks to imitate the way the human brain processes information. These models are capable of complex tasks such as image and speech recognition, natural language processing, and more.'}

In [135]:
# Accessing the output based on its structure
existing_answer = initial_response['text']  # Directly use the 'text' key for the refinement

In [136]:
# Step 2: Prepare new information (if any) for refinement
new_information = "AI technologies are rapidly advancing and are applied in various industries."

In [137]:
# Step 3: Run the Refine Chain with the initial response and new information
refine_inputs = {
    "existing_answer": existing_answer,  # Use the initial response output
    "new_information": new_information
}

In [138]:
# Invoke the refine chain
final_refined_response = refine_chain.invoke(input=refine_inputs)

In [139]:
final_refined_response

{'existing_answer': 'Artificial Intelligence (AI) is a field of computer science that aims to simulate human intelligence. Machine learning, a significant branch of AI, focuses on analyzing data and making predictions based on patterns. Deep learning models, a subset of machine learning, use neural networks to imitate the way the human brain processes information. These models are capable of complex tasks such as image and speech recognition, natural language processing, and more.',
 'new_information': 'AI technologies are rapidly advancing and are applied in various industries.',
 'text': 'Artificial Intelligence (AI) is a rapidly advancing field of computer science that aims to simulate human intelligence. Machine learning, a significant branch of AI, focuses on analyzing data and making predictions based on patterns. With AI technologies being applied in various industries, deep learning models have emerged as a subset of machine learning that use neural networks to imitate the way 